In [9]:
import json

In [10]:
import pyspark.sql.functions as F

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window

In [12]:
config = json.load(open("config.json"))

In [13]:
spark = SparkSession.builder\
    .config("spark.jars", config['JDBC_PATH']) \
    .config("spark.executor.extraClassPath", config['JDBC_PATH'])\
    .getOrCreate()

24/01/09 00:42:06 WARN Utils: Your hostname, przemyslaw-Latitude-E7450 resolves to a loopback address: 127.0.1.1; using 192.168.1.110 instead (on interface wlp2s0)
24/01/09 00:42:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/01/09 00:42:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/09 00:42:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [14]:
dim_calendar = spark.sql("""
            SELECT explode(sequence(DATE('2016-01-01'),DATE('2025-12-31'),INTERVAL 1 DAY)) AS calendar_full_date
        """) \
        .withColumn("calendar_id", F.row_number().over(Window.orderBy("calendar_full_date"))) \
        .withColumn("calendar_full_date_str", F.col("calendar_full_date").cast("string")) \
        .withColumn("calendar_year", F.year("calendar_full_date")) \
        .withColumn("calendar_quarter", F.quarter("calendar_full_date")) \
        .withColumn("calendar_month", F.month("calendar_full_date")) \
        .withColumn("calendar_day", F.day("calendar_full_date")) \
        .withColumn("calendar_weekday", F.dayofweek("calendar_full_date"))
        # TODO
        # .withColumn("calendar_weekday_str", F.???("calendar_full_date"))
        # .withColumn("calendar_is_month_end", F.???("calendar_full_date"))

In [15]:
dim_calendar.show()

24/01/09 00:42:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:42:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:42:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------------+-----------+----------------------+-------------+----------------+--------------+------------+----------------+
|calendar_full_date|calendar_id|calendar_full_date_str|calendar_year|calendar_quarter|calendar_month|calendar_day|calendar_weekday|
+------------------+-----------+----------------------+-------------+----------------+--------------+------------+----------------+
|        2016-01-01|          1|            2016-01-01|         2016|               1|             1|           1|               6|
|        2016-01-02|          2|            2016-01-02|         2016|               1|             1|           2|               7|
|        2016-01-03|          3|            2016-01-03|         2016|               1|             1|           3|               1|
|        2016-01-04|          4|            2016-01-04|         2016|               1|             1|           4|               2|
|        2016-01-05|          5|            2016-01-05|         2016|       

In [16]:
dbtable = "dim_calendar"

In [18]:
dim_calendar.write.format("jdbc")\
        .option("url",config['DST_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", "dim_calendar")\
        .option("user", config['DST_DB_USER'])\
        .option("password", config['DST_DB_PASSWORD'])\
        .save()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.
24/01/09 00:43:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:43:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:43:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:43:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:43:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partiti